Using the [Heart Attack Analysis & Prediction Dataset](https://www.kaggle.com/datasets/rashikrahmanpritom/heart-attack-analysis-prediction-dataset). This notebook is going to showcase how to use the MLXplain (OmniXAI) integration of [Fidex](https://hes-xplain.github.io/documentation/dimlpfidex/fidex/fidex/) & [FidexGloRules](https://hes-xplain.github.io/documentation/dimlpfidex/fidex/fidexglorules/).

First, the dataset must be pre-processed like the following:

In [1]:
import pandas as pd
import pathlib as pl
from omnixai.data.tabular import Tabular
from omnixai.visualization.dashboard import Dashboard
from omnixai.explainers.tabular.auto import TabularExplainer
from mlxplain.explainers.tabular.specific.dimlpfidex import DimlpBTModel


dataset = pd.read_csv("heart.csv")

cp = pd.get_dummies(
    dataset["cp"], prefix="cp", prefix_sep="_", columns=["cp"], dtype="int8"
)
dataset = pd.concat([dataset.iloc[:, :2], cp, dataset.iloc[:, 3:]], axis=1)
dataset = dataset.rename(
    columns={
        "cp_0": "cp_typical",
        "cp_1": "cp_atypical",
        "cp_2": "cp_nonanginal ",
        "cp_3": "cp_asymptomatic",
    }
)

output = pd.get_dummies(
    dataset["output"], prefix="risk", prefix_sep="_", columns=["output"], dtype="int8"
)

dataset = pd.concat([dataset.iloc[:, :-1], output], axis=1)
dataset = dataset.rename(columns={"risk_0": "risk_no", "risk_1": "risk_yes"})

/home/eldado/Documents/hepia/mlxplain/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Now we must split the dataset into a train and test subsets and write feature names in a file: 

In [2]:
dataset = dataset.sample(frac=1)
split = int(dataset.shape[0] * 0.8)

features = dataset.columns

nb_classes = 2
nb_features = len(features) - nb_classes

train_dataset = Tabular(
    data=dataset.iloc[:split,:]
)

test_dataset = Tabular(
    data=dataset.iloc[split:,:]
)

root_dir = pl.Path("out")
root_dir.mkdir(parents=True, exist_ok=True)
features_filename = "attributes.txt"

with open(root_dir.joinpath(features_filename), "w") as file:
    for feature in features:
        file.write(f"{feature}\n")

Let's train the [DimlpBT](https://hes-xplain.github.io/documentation/dimlpfidex/dimlp/dimlpbt/) model with our datasets:

In [3]:
# WARNING: verbose_console can be used but it is extremely slow when used in notebooks for some reason

model = DimlpBTModel(
    root_dir,
    train_dataset,
    test_dataset,
    nb_features,
    nb_classes,
    attributes_file=features_filename,
)

model.train()


0

The rule extraction must now be initialized and executed with Fidex & FidexGloRules to generate local & global explainations:

In [4]:
max_iterations = 10
min_covering = 2
max_failed_attempts = 15
min_fidelity = 1.0
lowest_fidelity_allowed = 0.9
use_minimal_version = True

explainer = TabularExplainer(
        explainers=["fidex", "fidexGloRules"],
        data=train_dataset,
        model=model,
        mode="classification",
        params={
            "fidex": {
                "max_iterations": max_iterations,
                "attributes_file": features_filename,
                "min_covering": min_covering,
                "max_failed_attempts": max_failed_attempts,
                "min_fidelity": min_fidelity,
                "lowest_min_fidelity": lowest_fidelity_allowed,
            },

            "fidexGloRules": {
                "heuristic": 1,
                "with_fidexGlo": True,
                "fidexGlo": {
                    "attributes_file": features_filename,
                    "with_minimal_version": use_minimal_version,
                    "max_iterations": max_iterations,
                    "min_covering": min_covering, 
                    "covering_strategy": True,
                    "max_failed_attempts": max_failed_attempts,
                    "min_fidelity": min_fidelity, 
                    "lowest_min_fidelity": lowest_fidelity_allowed, 
                },
                "attributes_file": features_filename,
                "nb_threads": 4,
                "with_minimal_version": use_minimal_version,
                "max_iterations": max_iterations,
                "min_covering": min_covering,
            }
        },
)
local_explainations = explainer.explain(X=test_dataset)
global_explainations = explainer.explain_global()

dashboard = Dashboard(local_explanations=local_explainations, global_explanations=global_explainations)
dashboard.show()

From autoexplainer before lime (61, 18)


[2024-08-27 15:14:35,612] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/eldado/Documents/hepia/mlxplain/.venv/lib/python3.10/site-packages/flask/app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/eldado/Documents/hepia/mlxplain/.venv/lib/python3.10/site-packages/flask/app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/eldado/Documents/hepia/mlxplain/.venv/lib/python3.10/site-packages/flask/app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/eldado/Documents/hepia/mlxplain/.venv/lib/python3.10/site-packages/flask/app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "/home/eldado/Documents/hepia/mlxplain/.venv/lib/python3.10/site-packages/dash/dash.py", line 1373, in dispatch
    ctx.run(
  File "/home/elda